In [3]:
import geopandas as gpd
import os
import fiona
import ipywidgets as widgets
from IPython.display import display
from rasterstats import zonal_stats
import rasterio
import rasterio.fill
from shapely.geometry import shape, mapping
import json
from earthpy import clip
import earthpy.spatial as es
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
import gdal
import datetime
import warnings
import pandas as pd
import scipy.spatial
from pandas import DataFrame as df
from shapely.geometry import Point, Polygon
warnings.filterwarnings('ignore')

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

''

# Part 1: Convert csv to shapefile 

In [29]:
path = r"C:\Somalia\Agrodem_output"
name_of_file= "test.csv"
# Import csv as pandas dataframe
df_somalia= pd.read_csv(path + "\\" + name_of_file)

In [30]:
new = df(df_somalia,columns =['Pixel', 'state', 'lon', 'lat', 'Crop','Annual_elec_demand'])

In [31]:
new.rename(columns={"state":"name_adm2"}, inplace=True)

In [32]:
new.head()

,Pixel,name_adm2,lon,lat,Crop,Annual_elec_demand
0,1,Baki,43.375000,10.041667,Sorghum,232890.04880
1,2,Diinsoor,42.791667,2.541667,Sorghum,27352.96171
2,3,Diinsoor,42.875000,2.541667,Sorghum,289526.71560
3,4,Diinsoor,42.958333,2.541667,Sorghum,413781.02190
4,5,Diinsoor,43.041667,2.541667,Sorghum,112429.62900


In [33]:
# Add geometry and convert to spatial dataframe in source CRS
new['geometry'] = list(zip(new['lon'], new['lat']))
new['geometry'] = new['geometry'].apply(Point)
new = gpd.GeoDataFrame(new,geometry='geometry', crs={'init': 'epsg:4326'})

# Reproject data in to Ordnance Survey GB coordinates
new_osgb = new.to_crs({'init': 'epsg:3395'})

In [34]:
# save
path = r"C:\Somalia\Agrodem_output_postprocessing"
#csv:
shpname = "crop_irrigation_demand_Somalia.csv"
new_osgb.to_csv(os.path.join(path,"{c}".format(c=shpname)), index=False)
#shp:
new_osgb.to_file('crop_irrigation_demand_Somalia.shp',driver = 'ESRI Shapefile')
# Define output path

# Part 2: Select the workspace  and admin boundaries

In [35]:
messagebox.showinfo('OnSSET extraction', 'Output folder')
workspace = filedialog.askdirectory()

messagebox.showinfo('OnSSET', 'Select the admin boundaries')
admin = gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))

## Select the clusters

In [36]:
crs = 'EPSG:3395'

In [37]:
messagebox.showinfo('OnSSET', 'Select the clusters')
clusters = gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
    
popunit = widgets.Dropdown(options=clusters.head(),
    value=None,
    description='Population:',
    disabled=False)
display(popunit)
x = popunit.value

Dropdown(description='Population:', options=('Country', 'NightLight', 'Pop', 'GridCellAr', 'ElecPop', 'id', 'A…

In [38]:
def preparing_for_vectors(workspace, clusters, crs):   
    clusters.crs = {'init' :'epsg:4326'}
    clusters = clusters.to_crs({ 'init': crs}) 
    points = clusters.copy()
    points["geometry"] = points["geometry"].centroid
    points.to_file(workspace + r'\clusters_cp.shp', driver='ESRI Shapefile')
    print(datetime.datetime.now())    
    return clusters

In [39]:
clusters = preparing_for_vectors(workspace, clusters, crs)

2020-09-29 11:22:37.256285


In [40]:
messagebox.showinfo('OnSSET', 'Select the irrigation demand map')
irrdem=gpd.read_file(filedialog.askopenfilename(title = "Select Irrigation Demand map", filetypes = (("shapefile","*.shp"),("all files","*.*"))))

irrdemand = widgets.Dropdown(options=irrdem.head(),
    value=None,
    description='IrrigationDemand:',
    disabled=False)

display(irrdemand)
      
IrrDemandunit = widgets.Dropdown(options=['Wh', 'kWh', 'MWh'],
    value='kWh',
    description='Unit:',
    disabled=False)

display(IrrDemandunit)

Dropdown(description='IrrigationDemand:', options=('Pixel', 'name_adm2', 'lon', 'lat', 'Crop', 'Annual_ele', '…

Dropdown(description='Unit:', index=1, options=('Wh', 'kWh', 'MWh'), value='kWh')

In [43]:
def get_irrigation_and_residential_clusters(admin, points, crs): 
    points_clip = gpd.clip(points, admin)
    points_clip.crs = {'init':'epsg:4326'}
    points_proj=points_clip.to_crs({'init': crs})

    #points_proj.to_file(workspace + r"\IrrDemandDist_proj.shp", driver='ESRI Shapefile')
    points_f = fiona.open(r"C:\Users\adm.esa\Documents\GitHub\agrodem_new\crop_irrigation_demand_Somalia.shp")
    points2 = fiona.open(workspace + r'\clusters_cp.shp')

    geoms1 = [shape(feat["geometry"]) for feat in points_f]
    s1 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms1]
    irrigation_cluster = np.array(s1) # Irrigation Demand
    
    geoms2 = [shape(feat["geometry"]) for feat in points2]
    s2 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms2]
    residential_cluster = np.array(s2) # Residential 
    return irrigation_cluster, residential_cluster 

def map_irrigation_demand_to_residential_cluster_point(irrigation_cluster, residential_cluster):
    """
    
    """
    #:: step1. build tree with residential_cluster point
    residential_demand_kd_tree = scipy.spatial.cKDTree(residential_cluster)
    #:: step2. query residential demand tree with irrigation cluster. This would give K-points 
    #:: that are closest to each point in irrigation cluster
    dist, indexes = residential_demand_kd_tree.query(irrigation_cluster, k = 1)   
    return np.array(dist),np.array(indexes)
        
def processing_irrdemand(admin, crs, workspace, clusters, points, IrrDemandValue, 
                     IrrDemandunit):

    #:: read irrigation and residential clusters 
    irrigation_cluster, residential_cluster = get_irrigation_and_residential_clusters(admin, points, crs)
    #:: 
    #:: Indexes : list of integers representing the index into the residential_cluster to which the irrigation 
    #::    cluster point has been assigned. 
    #:: 
    dist, indexes = map_irrigation_demand_to_residential_cluster_point(irrigation_cluster, residential_cluster)
    """
     Tabl 
     Resid-Id | Annual
    """
    z1=dist.tolist()
    z2=indexes.tolist()
    print (z2,z1)
    #create an empty dataframe to fill the new irrigation_table into
    irrigation_df= pd.DataFrame()
    #creating the residential ID column which the irrigation_demand clusters have mapped to
    irrigation_df['id']  = z2
    irrigation_df['IrrDistance']=z1
    #creating  the irrigation demand column
    z3 = []
    for s in range(len(z2)):
        z3.append(points[IrrDemandValue][s])
    print ("z3 is:", z3)
    irrigation_df['IrrDemand'] =z3
    print("irrigation_df is:", irrigation_df)
    new_df = irrigation_df.groupby('id').sum()
    #PRNT AND VERIFY THIS. 
    print("new_df:", new_df)
   
    
    x = IrrDemandunit
    
    if x is 'MWh':
        irrigation_df['IrrDemand'] = irrigation_df['IrrDemand']/1000
    elif x is 'kWh':
        irrigation_df['IrrDemand'] = irrigation_df['IrrDemand']
    else:
        irrigation_df['IrrDemand'] = irrigation_df['IrrDemand']/1000000


    new_clusters = pd.merge(clusters,new_df,on ="id",how="left")
    print(clusters)
    print(new_clusters)
 

    print(datetime.datetime.now())
    
    return new_clusters

In [44]:
new_clusters = processing_irrdemand(admin, crs, workspace, clusters, irrdem, irrdemand.value, IrrDemandunit.value)

[17412, 24268, 24269, 23853, 24293, 23986, 23965, 24157, 24155, 23954, 23971, 24038, 24052, 24332, 24272, 23850, 24033, 24352, 24359, 24190, 24208, 24542, 24537, 23840, 23836, 23871, 23885, 23924, 25336, 25316, 25212, 25460, 25460, 25287, 25214, 25262, 25489, 25460, 25287, 25214, 25399, 25512, 25407, 25475, 25446, 25555, 25452, 25556, 25557, 25534, 25533, 25391, 25388, 25435, 53277, 54680, 54672, 53289, 53296, 54717, 54886, 54851, 53392, 53268, 53264, 25054, 25230, 25201, 25204, 25296, 25302, 25344, 25336, 25312, 25041, 25664, 25084, 25640, 25685, 25678, 25655, 25603, 25241, 26785, 26785, 26796, 26796, 26790, 26797, 26289, 26257, 26257, 26269, 26257, 26257, 26230, 26437, 26440, 26438, 26437, 26311, 26311, 26435, 26235, 26239, 26238, 26311, 26240, 26237, 26657, 26658, 26335, 26660, 26315, 26317, 26319, 26661, 26318, 26305, 26305, 26649, 26105, 26700, 26694, 26011, 26537, 26414, 26411, 26596, 25974, 26048, 25947, 26775, 26360, 26473, 25749, 26567, 26849, 26927, 26849, 26242, 26924, 26418

In [45]:
new_clusters["X_deg"] = new_clusters.geometry.centroid.x
    
new_clusters["Y_deg"] = new_clusters.geometry.centroid.y

new_clusters.fillna(0,inplace=True)

In [46]:
new_clusters.to_file(workspace + r"\clusters_with_AgriDemand.shp", driver='ESRI Shapefile')
new_clusters.to_file(workspace + r"\clusters_with_AgriDemand.csv", driver='CSV')